## Blockchain project

In this project the student will write code to create a simple blockchain.

The ingredients of a blockchain are a hash linked list and proof of work.

We explain both concepts below, and then give the definition of a blockchain.

We assume that we have a hash function H that satisfies the properties of a secure hash, namely

* Collision resistance
* Preimage resistance
* Second preimage resistance


### Hash linked list

One of the basic ingredients in a blockchain is a hash linked list.

Imagine a series of messages $M_0,M_1,\ldots,M_n$ where $M_i$ contains a hash of $M_{i-1}$.

Then if $M_i$ is altered, the hashes in $M_{i+1},M_{i+2},\ldots,M_n$ must be recomputed.


### Proof of work

Another basic ingredient of a blockchain is proof of work.  

**Example:**  You are asked to provide a file $F$ such that $H(F)$ begins with 24 zero bits.

If H is secure, the only way to do this is brute force random search.

You can include in F a number (called a nonce) that is adjusted until the condition is met.

Note that F may also contain other information.

Thus H(F) proves that a certain amount of work has been done to find $F$  (about $2^{24}$ hashes in this example).


### A real blockchain

Consider a hash linked list in which each message contains a nonce.

The nonces are adjusted so that $H(M_i)$ always begins with 24 zero bits for each $i=1,2,\ldots,n$.

Now if $M_i$ is altered, the hashes in $M_{i+1},M_{i+2},\ldots,M_n$ must be recomputed.

But now this takes a lot of work, because $H(M_i)$ must be small and the corresponding nonces must be found through brute force search.

If enough work is required and the chain is long enough, it is infeasible to ever modify the deep parts of the chain.




In [98]:
"""
This is a warm up with the basic operations in python needed to create a blockchain.
In other words, we show how to hash in python. 
(provided to the student)

We make a "block" such as we might have in a blockchain.

For example:

Data: Hunter Johnson
Nonce: 23844253449463976384357805956163882646
PrevHash: 0

Then we hash it.

We also write the block to a file and hash from the command line.
This makes sure that the hash of the block (as a string) is the same
as the hash of the block (as a file). 

"""

import hashlib

import random
dataline="Data: Hunter Johnson\n"
nonceline = "Nonce: {}\n".format(random.randint(0,2**128))
prevhash = 0
prevhashline = "PrevHash: {}\n".format(prevhash)

string2hash = dataline+nonceline+prevhashline
print("Hashing this data:\n\n{}".format(string2hash))
string2hash = string2hash.encode()
h=hashlib.sha256()
h.update(string2hash)
D=h.digest()
print("{}".format(D.hex()))


print("\nWriting same data to file...")
fp = open("testblock.txt","w")
fp.write(string2hash.decode())
fp.close()


## The following assumes that sha256sum is installed!
print("Hash of the file using sha256sum on command line (should be the same as the above):\n")

!sha256sum testblock.txt | awk '{print $1}'


Hashing this data:

Data: Hunter Johnson
Nonce: 23844253449463976384357805956163882646
PrevHash: 0

09cf18f7f76ef45a51cf2912358c52bf8490855b68d5d42d6f54f835a80e9af7

Writing same data to file...
Hash of the file using sha256sum on command line (should be the same as the above):

09cf18f7f76ef45a51cf2912358c52bf8490855b68d5d42d6f54f835a80e9af7


In [111]:
"""Takes: data, nonce, and previous hash
   Returns: an encoded string of the form
   
   Data: <data>
   Nonce: <nonce>
   PrevHash: <prevhash>
   
   The string ends in a newline.
"""
def  makeblock(data,nonce,prevhash):
    dataline="Data: {}\n".format(data)
    nonceline = "Nonce: {}\n".format(nonce)
    prevhashline = "PrevHash: {}\n".format(prevhash)
    string2hash = dataline+nonceline+prevhashline
    return string2hash.encode()

"""Takes: An encoded string of the form returned by makeblock
   Returns: The sha256 digest of the string as a hex string
"""
def hashblock(block):
    h=hashlib.sha256()
    h.update(block)
    D=h.digest()
    return D.hex()

import math

"""Takes: The data portion of a block, the hash of the previous block, and the desired difficulty.
   Returns: The output of makeblock for the given data and prevhash, with appropriate nonce. 
            The block must meet the given difficulty requirement (ie have a small hash).
   Also returned: The hash of the block returned.
   
   Meaning of "difficulty": Let N be the numerical value of the hash of the block.
   Then the nonce is selected so that the base-2 logarithm of N is <= difficulty.
"""
def mineblock(data,prevhash,difficulty=232):
    hbnum = 2**256
    while math.log(hbnum,2)>difficulty:
        r = random.randint(0,2**80)
        #
        # Put code here that tries to mine the block
        #
        #
    return block,hb

"""Takes: A filename and a block as an encoded string.
   Returns: Nothing
   Effect: writes the block *as text* to the named file.
"""
def writeblock(filename,block):
    fp = open(filename,"w")
    fp.write(block.decode())
    fp.close()  
    
    
"""Takes: A filename
   Returns: the text inside the file *as text*.
"""    
def readblock(filename):
    fp=open(filename,"r")
    block = fp.read()
    fp.close()
    return block

In [120]:
"""
This creates a sample blockchain with the default difficulty and silly messages.

Depending on the difficulty level this may take a long time.

It takes on the order of 5 minutes on my machine.

The blockchain files are created in the current directory.

"""

def makechain():
    block,blockhash=mineblock("<your name here>",0)
    writeblock("B1.txt",block)
    block,blockhash=mineblock("Financial Weirdness",blockhash)
    writeblock("B2.txt",block)
    block,blockhash=mineblock("I'm a nutjob",blockhash)
    writeblock("B3.txt",block)
    block,blockhash=mineblock("Buy ravens now while you can!",blockhash)
    writeblock("B4.txt",block)
    block,blockhash=mineblock("Invest in used cars",blockhash)
    writeblock("B5.txt",block)
    
%time makechain()    

CPU times: user 5min 30s, sys: 5.24 ms, total: 5min 30s
Wall time: 5min 31s


In [0]:
"""  ## The time to run the C++ version
real	29m1.781s
user	29m1.650s
sys	0m0.016s
"""
pass

In [109]:
"""
This function validates a blockchain by verifying that 

1) The chain forms a hash linked list, and
2) The hashes meet the difficulty specification

The variable "chain" is a list of filenames, which give the blockchain blocks in proper order.

"""

def validate(chain,difficulty=232):
    if len(chain)==0:
        return True
    genesis=chain[0]
    gblock = readblock(genesis)
    prevhash = hashblock(gblock.encode())
    if math.log(int(prevhash,16),2)>difficulty:
        print("Difficulty of {} not met in block\n{}".format(difficulty,gblock))
        return False
    for bfile in chain[1:]:
        block = readblock(bfile)
        loc=block.find("PrevHash: ")
        claimedprevhash=block[loc+len("PrevHash: "):loc+len("PrevHash: ")+64]
        if prevhash != claimedprevhash:
            print("Chain breaks at {}".format(bfile))
            print("Claimed prev hash is {} but prev hash is {}".format(claimedprevhash,prevhash))
            return False
        prevhash=hashblock(block.encode())
        if math.log(int(prevhash,16),2)>difficulty:
            print("Difficulty of {} not met in block {}".format(difficulty,gblock))
            return False
    return True

In [117]:
validate(["C1.txt","C2.txt","C3.txt","C4.txt","C5.txt"])

Chain breaks at C5.txt
Claimed prev hash is 000001fb1087ec2cfaca9f916204a9b49a59aaf85dd49679d991580cab2c76f7 but prev hash is 0000005a314b0029b9d7547dbbe34baa0389fda9eea78f5c512a93c8d1879cb8


False